# Week 2

During week 2 of the project, we added some moving obstacles and the dino in the form of some boxes. We then created a Python GUI to interact with the game. The GUI has 3 buttons:
- Start button to start the game
- Jump button to make the dino jump
- Restart button to restart the game in case the dino dies

We also used the GUI for debugging purposes.

Once we had our GUI ready, we implemented the jump as well as the collision logic in our code. Once it was up and running, we implemented the voice control feature using ADC input and FFT algorithm.

---

## The Design

The code was implemented in various phases as described below.

#### 1. Implementing the Obstacles

Just like with implementing the ground, we implemented the obstacles using a `struct`. However, there is a catch. The ground was implemeted using an array while we only used single variable to implement the obstacle (and reuse it to bring in the next obstacle). We also used a new variable called `obsType` to keep a track of the type of obstacle (we have 3 different types based on their width and height).

```c
#define OBS_W_0 10
#define OBS_H_0 20
#define OBS_W_1 15
#define OBS_H_1 30
#define OBS_W_2 30
#define OBS_H_2 20

int obsType;

struct Obstacle{
    int x, w, h;
};
struct Obstacle obstacle;
```

<br>

We initialized the obstacle in main as below:

```c
obsType = randomRange(0, 3);
obstacle.x = WIDTH;
if(obsType == 0){
    obstacle.w = OBS_W_0;
    obstacle.h = OBS_H_0;
}
else if(obsType == 1){
    obstacle.w = OBS_W_1;
    obstacle.h = OBS_H_1;
}
else{
    obstacle.w = OBS_W_2;
    obstacle.h = OBS_H_2;
}
```

<br>

Now that we had initialized the obstacle, we needed to update it for every iteration. It was done so by following a simple algorithm:
- Update the x-coordinate of the obstacle based on the speed
- If the obstacle has reached the left-most edge, reset its parameters

This algorithm has been implemented as below:

```c
tft_fillRect(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obstacle.w, obstacle.h, ILI9340_BLACK);

obstacle.x -= SPEED;
if(obstacle.x + obstacle.w < 0){
    obsType = randomRange(0, 3);
    obstacle.x = WIDTH + randomRange(0, 50);
    switch(obsType){
        case 0: obstacle.w = OBS_W_0;
                obstacle.h = OBS_H_0;
                break;
        case 1: obstacle.w = OBS_W_1;
                obstacle.h = OBS_H_1;
                break;
        case 2: obstacle.w = OBS_W_2;
                obstacle.h = OBS_H_2;
                break;
    }
}

tft_fillRect(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obstacle.w, obstacle.h, HARD_COLOR); 
```

<br>

The output obtained after running this code is given below.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/Week2Obstacle.jfif" style="width: 446px; height: 273px;" >
</div>
<figure>
    <center><figcaption>The Moving Obstacle Frame</figcaption></center>
</figure>

#### 2. Implementing the Dino

Now that we had implemented the obstacle, we had to implement the Dino. Just like with implementing the obstacke, we implemented the dino using a `struct`. The dino is a little different from the ground and obstacles. Its x-coordinate is to remain constant and the y-coordinate is to change. It also needs to have a flag in order to keep track of whether the dino is alive or dead and a parameter to keep track of its velocity when it jumps. It is implemented as below.

```c
struct Player{
    int x, y, w, h, vy;
    int alive;
};
struct Player myPlayer;
```

<br>

We initialized the dino (`myPlayer`) in main as below.

```c
myPlayer.x = 30;
myPlayer.y = 0;
myPlayer.w = 22;
myPlayer.h = 25;
myPlayer.vy = 0;
myPlayer.alive = 1;
```

<br>

Now that we had initialized the dino, we needed to update it for every iteration. It was done so by simply clearing old position of the dino and drawing the new position. The jump logic was implemented later on.

```c
tft_fillRect(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), myPlayer.w, myPlayer.h, ILI9340_BLACK);
tft_fillRect(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), myPlayer.w, myPlayer.h, SOFT_COLOR);
```

<br>

The output obtained after running this code is given below.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/Week2Dino.jfif" style="width: 446px; height: 273px;" >
</div>
<figure>
    <center><figcaption>Implementation of the Dino</figcaption></center>
</figure>

#### 3. Implementing the GUI

In order to create the GUI, we used the [sample code](https://people.ece.cornell.edu/land/courses/ece4760/PIC32/remote/target4/pic_target_4a.txt) provided by Prof. Bruce Land for the Lab 0 of ECE 4760 course and modified it in order to create our GUI. In particular, we changed the `layout` list to add 3 buttons to be used. We also had to bind the corrosponding `ButtonRelease` functionality.

```python
layout = [[sg.RealtimeButton('START!', key = 'pushbut01', font = 'Helvetica 12'),
           sg.RealtimeButton('JUMP!', key = 'pushbut02', font = 'Helvetica 12'),
           sg.RealtimeButton('RESTART!', key = 'pushbut03', font = 'Helvetica 12')],
            
          [sg.Text('Serial data to PIC', background_color = heading_color)],
          [sg.InputText('', size = (40, 10), key = 'pic_input', do_not_clear = False, enable_events = False, focus = True),
           sg.Button('Send', key = 'pic_send', font = 'Helvetica 12')],
           
          [sg.Text('Serial data from PIC', background_color = heading_color)],
          [sg.Multiline('', size = (50, 10), key = 'console', autoscroll = True, enable_events = False)],
          
          [sg.Text('System Controls', background_color = heading_color)],
          [sg.Button('Exit', font = 'Helvetica 12')],
          [sg.Checkbox('reset_enable', key = 'r_en', font = 'Helvetica 8', enable_events = True),
           sg.Button('RESET PIC', key = 'rtg', font = 'Helvetica 8')
         ]]

window['pushbut01'].bind('<ButtonRelease-1>', 'r')
window['pushbut02'].bind('<ButtonRelease-1>', 'r')
window['pushbut03'].bind('<ButtonRelease-1>', 'r')
```

The complete code for the python GUI can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/Week2PythonCode.txt). The generated GUI looks as shown below.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/Week2GUI.png" style="width: 450px; height: 477px;" >
</div>
<figure>
    <center><figcaption>The Python GUI</figcaption></center>
</figure>